In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\JAGADEESHNAIKPALTHYA\langgraph-end-to-end\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [5]:
llm.invoke("hi,how are you?")

AIMessage(content="I'm doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you may have. How can I help you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 15, 'total_tokens': 63, 'completion_time': 0.087272727, 'prompt_time': 0.001900817, 'queue_time': 0.23436453200000001, 'total_time': 0.089173544}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0de4afd-6bf8-4601-91f2-a140b7cb4daf-0', usage_metadata={'input_tokens': 15, 'output_tokens': 48, 'total_tokens': 63})

### Pre-difined Tools

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query": "langchan"}))

No good Wikipedia Search Result was found


In [7]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("sunny savita")

"['https://www.youtube.com/watch?v=cydntJObPLI&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=6GnsndNL5u4&pp=ygUMc3Vubnkgc2F2aXRh']"

In [8]:
from langchain_community.tools import YouTubeSearchTool
tool2=YouTubeSearchTool()
tool2.run("carry minati")

"['https://www.youtube.com/watch?v=m9s1NQG3TNY&pp=ygUMY2FycnkgbWluYXRp', 'https://www.youtube.com/watch?v=GOFQN8otiYs&pp=ygUMY2FycnkgbWluYXRp']"

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'title': 'Burning Man flooding: What happened to stranded festivalgoers?',
  'url': 'https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331',
  'content': '2:49\nAttendees look at a double rainbow over flooding on a desert plain at the Burning Man festival in Black Rock City, Nevada, September 1, 2023.\nJulie Jammot/AFP via Getty Images\nTens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend. [...] Camps are set on a muddy desert plain at the Burning Man festival in Black Rock, City, Nevada, Sept. 2, 2023.\nJulie Jammot/AFP via Getty Images\nWhat happened?\nBlack Rock Desert was faced with two to three months worth of rain in just a matter of hours on Friday, Sept. 1.\nOn average, the area gets only 0.2 inches of rain or less in September -- but festivalgoers were met with up to 1 inch in some areas, in a desert that gets only about 5 to 6 inches

### Custom tools

In [10]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [11]:
get_word_length.invoke('Hii')

3

In [12]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [13]:
@tool
def summisation(a: int, b: int) -> int:
    """Adding two numbers."""
    return a + b

In [14]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### Agents

In [15]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [16]:
tool=load_tools(["wikipedia"],llm=llm)

In [17]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\JAGADEESHNAIKPALTHYA\AppData\Local\Temp\ipykernel_14780\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [18]:
agent.run('who is keerthi suresh')

C:\Users\JAGADEESHNAIKPALTHYA\AppData\Local\Temp\ipykernel_14780\895738560.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run('who is keerthi suresh')




> Entering new AgentExecutor chain...
Thought: I need to look up Keerthi Suresh on Wikipedia to find information about her.
Action: wikipedia
Action Input: Keerthi Suresh
Observation: Page: Keerthy Suresh
Summary: Keerthy Suresh (born 17 October 1992) is an Indian actress who appears predominantly in  Tamil and Telugu films. Her accolades include one National Film Award, five SIIMA Awards and two Filmfare Awards South. Keerthy was placed in Forbes India's 30 Under 30 list of 2021. 
Keerthy is the daughter of film producer G. Suresh Kumar and actress Menaka. She began her career as a child actress in the early 2000s and returned to films after studying fashion design. She had her first lead role in the 2013 Malayalam film Geethaanjali, for which she won the SIIMA Award for Best Female Debut – Malayalam. After winning the SIIMA Award for Best Female Debut – Tamil for Idhu Enna Maayam (2015), Keerthy went on to star in successful films such as Ring Master (2014), Nenu Sailaja (2016), Ra

'Keerthi Suresh is an Indian actress who predominantly works in Tamil and Telugu films.'

In [19]:
agent.run("what is llama and who create this llm model?")



> Entering new AgentExecutor chain...
Thought: I need to look up "llama" on Wikipedia to see if it refers to the large language model and who created it.
Action: wikipedia
Action Input: llama large language model
Observation: Page: Llama (language model)
Summary: Llama (Large Language Model Meta AI, formerly stylized as LLaMA) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 3.3, released in December 2024.
Llama models are trained at different parameter sizes, ranging between 1B and 405B. Initially  only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.
Model weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under license

'Llama is a family of large language models (LLMs) released by Meta AI.'

In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [21]:
search = TavilySearchResults()

In [22]:
search.invoke("what is the weather in SF")

[{'title': 'Weather in San Francisco',
  'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743588079, 'localtime': '2025-04-02 03:01'}, 'current': {'last_updated_epoch': 1743588000, 'last_updated': '2025-04-02 03:00', 'temp_c': 10.4, 'temp_f': 50.7, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 330, 'wind_dir': 'NNW', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 68, 'cloud': 25, 'feelslike_c': 8.2, 'feelslike_f': 46.7, 'windchill_c': 6.9, 'windchill_f': 44.5, 'heatindex_c': 9.3, 'heatindex_f': 48.7, 'dewpoint_c': 6.1, 'dewpoint_f': 43.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 15.8, 'gust_kph': 25.5}}",
  '

In [23]:
tools = [search]

In [24]:
from langchain import hub

In [25]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [27]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [28]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [29]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [30]:
agent_executor.invoke({"input": "What is the corrent GDP of India"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'What is the current GDP of India?'}`


[{'title': 'India GDP - Worldometer', 'url': 'https://www.worldometers.info/gdp/india-gdp/', 'content': 'Population\nMore \nWGDPGDP by Country India GDP\n India GDP\nGDP nominal (2022)\n$3,385,090,000,000\n(Current US Dollars). Sources include: World Bank, United Nations.\nGDP growth rate (2022)\n7.00% \nNominal (current) Gross Domestic Product (GDP) of India is $3,385,090,000,000 (USD) as of 2022.\nReal GDP (constant, inflation adjusted) of India reached $2,432,020,000,000 in 2022. [...] GDP Growth Rate in 2022 was 7.00%, representing a change of 193,390,000,000 US$ over 2021, when Real GDP was $2,761,590,000,000.\nGDP per Capita in India (with a population of 1,425,423,212 people) was $2,073 in 2022, an increase of $120 from $1,953 in 2021; this represents a change of 6.2% in GDP per capita.\nCountryContinentWorld\nIndia GDP (Nominal, $USD) 2003-2022

{'input': 'What is the corrent GDP of India', 'output': '\n'}

In [31]:

print(agent_executor.invoke({"input": "whats the weather in new york"}))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in new york'}`


[{'title': 'Weather in New York', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1743584666, 'localtime': '2025-04-02 05:04'}, 'current': {'last_updated_epoch': 1743584400, 'last_updated': '2025-04-02 05:00', 'temp_c': 3.9, 'temp_f': 39.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14.8, 'wind_degree': 39, 'wind_dir': 'NE', 'pressure_mb': 1029.0, 'pressure_in': 30.38, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 35, 'cloud': 0, 'feelslike_c': 0.4, 'feelslike_f': 32.8, 'windchill_c': -1.8, 'windchill_f': 28.8, 'heatindex_c': 1.7, 'heatindex_f': 35.0, 'dewpoint_c': -2.4, 'dewpoint_f': 2

In [32]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [33]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [35]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [38]:
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

In [40]:
retriever.invoke("what is longsmit")[0]

Document(id='65821685-2f48-46d8-ae4c-98613108c89b', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze traces in LangSmith

In [41]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [43]:
tools = [search, retriever_tool]

In [44]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [45]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [46]:
agent_executor.invoke({"input": "hi! what is a langsmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'what is a langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development -

{'input': 'hi! what is a langsmith?',
 'output': 'LangSmith is a platform for building production-grade LLM applications.'}

In [47]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in sf'}`


[{'title': 'Weather in san francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743589335, 'localtime': '2025-04-02 03:22'}, 'current': {'last_updated_epoch': 1743588900, 'last_updated': '2025-04-02 03:15', 'temp_c': 8.9, 'temp_f': 48.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 330, 'wind_dir': 'NNW', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 71, 'cloud': 25, 'feelslike_c': 6.3, 'feelslike_f': 43.4, 'windchill_c': 6.9, 'windchill_f': 44.5, 'heatindex_c': 9.3, 'heatindex_f': 48.7, 'dewpoint_c': 6

{'input': 'whats the weather in sf?',
 'output': '```json\n{"tool_call": \n"tool_call":"tool_call":{"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_call":"tool_\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n\n\n\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n